# Tarea #: 3

## Tema: Clasificación de datos utilizando imágenes

In [ ]:
import glob
import cv2
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
def cargar_imagenes_y_labels(ruta_base, tamaño=(64, 64)):
    X = []
    y = []
    clases = sorted([d for d in os.listdir(ruta_base) if os.path.isdir(os.path.join(ruta_base, d))])
    clase_dict = {clase: idx for idx, clase in enumerate(clases)}
    print("Clases encontradas:", clase_dict)

    for clase in clases:
        ruta_clase = os.path.join(ruta_base, clase)
        archivos = glob.glob(os.path.join(ruta_clase, "*"))
        print(f"Cargando {len(archivos)} imagenes de la clase '{clase}'")

        for archivo in archivos:
            img = cv2.imread(archivo)
            if img is not None:
                img = cv2.resize(img, tamaño)
                X.append(img)
                y.append(clase_dict[clase])
            else:
                print(f"No se pudo leer la imagen: {archivo}")

    print(f"Total imagenes cargadas: {len(X)}")
    return np.array(X, dtype=np.float32) / 255.0, np.array(y), clase_dict

In [9]:
X, y, clase_dict = cargar_imagenes_y_labels("data/train")

Clases encontradas: {'Black-grass': 0, 'Charlock': 1, 'Cleavers': 2, 'Common Chickweed': 3, 'Common wheat': 4, 'Fat Hen': 5, 'Loose Silky-bent': 6, 'Maize': 7, 'Scentless Mayweed': 8, 'Shepherds Purse': 9, 'Small-flowered Cranesbill': 10, 'Sugar beet': 11, 'cat': 12, 'cow': 13, 'dog': 14, 'horse': 15}
Cargando 263 imágenes de la clase 'Black-grass'
Cargando 390 imágenes de la clase 'Charlock'
Cargando 287 imágenes de la clase 'Cleavers'
Cargando 611 imágenes de la clase 'Common Chickweed'
Cargando 221 imágenes de la clase 'Common wheat'
Cargando 475 imágenes de la clase 'Fat Hen'
Cargando 654 imágenes de la clase 'Loose Silky-bent'
Cargando 221 imágenes de la clase 'Maize'
Cargando 516 imágenes de la clase 'Scentless Mayweed'
Cargando 231 imágenes de la clase 'Shepherds Purse'
Cargando 496 imágenes de la clase 'Small-flowered Cranesbill'
Cargando 385 imágenes de la clase 'Sugar beet'
Cargando 164 imágenes de la clase 'cat'
Cargando 164 imágenes de la clase 'cow'
Cargando 164 imágenes d

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_flat = X_train.reshape(len(X_train), -1)
X_val_flat = X_val.reshape(len(X_val), -1)


In [ ]:
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train_flat, y_train)

y_pred_lr = model_lr.predict(X_val_flat)
acc_lr = accuracy_score(y_val, y_pred_lr)
print("Accuracy regresion logistica:", acc_lr)

In [ ]:
y_pred_lr = model_lr.predict(X_val_flat)

cm_lr = confusion_matrix(y_val, y_pred_lr)
ConfusionMatrixDisplay(cm_lr).plot()
plt.title("Matriz de confusion regresion logistica")
plt.show()


In [ ]:
y_train_cat = to_categorical(y_train, num_classes=len(clase_dict))
y_val_cat = to_categorical(y_val, num_classes=len(clase_dict))

cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(clase_dict), activation='softmax')
])

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn.fit(X_train, y_train_cat, epochs=10, validation_data=(X_val, y_val_cat))

In [ ]:
y_pred_cnn = cnn.predict(X_val)
y_pred_cnn_labels = np.argmax(y_pred_cnn, axis=1)

cm_cnn = confusion_matrix(y_val, y_pred_cnn_labels)
ConfusionMatrixDisplay(cm_cnn).plot()
plt.title("Matriz de confusion CNN")
plt.show()

In [ ]:
def cargar_imagenes_test(ruta_test, tamaño=(64, 64)):
    X_test = []
    nombres = []
    archivos = sorted(glob.glob(os.path.join(ruta_test, "*")))
    print(f"imagenes en test: {len(archivos)}")

    for archivo in archivos:
        img = cv2.imread(archivo)
        if img is not None:
            img = cv2.resize(img, tamaño)
            X_test.append(img)
            nombres.append(os.path.basename(archivo))
        else:
            print(f"No se pudo leer la imagen: {archivo}")

    X_test = np.array(X_test, dtype=np.float32) / 255.0
    return X_test, nombres


In [25]:
X_test, test_names = cargar_imagenes_test("data/test")

Total imágenes en test: 962


In [28]:
X_test_flat = X_test.reshape(len(X_test), -1)
y_test_pred_lr = model_lr.predict(X_test_flat)

idx_to_class = {v: k for k, v in clase_dict.items()}
labels_lr = [idx_to_class[i] for i in y_test_pred_lr]

# Exportar CSV
import pandas as pd

df_lr = pd.DataFrame({
    "file": test_names,
    "label": labels_lr
})
df_lr.to_csv("predicciones_logistica.csv", index=False)
print("generado correctamente")


✅ CSV de regresión logística guardado como 'predicciones_logistica.csv'


In [29]:
y_test_pred_cnn = cnn.predict(X_test)
y_test_pred_cnn_labels = np.argmax(y_test_pred_cnn, axis=1)

labels_cnn = [idx_to_class[i] for i in y_test_pred_cnn_labels]

df_cnn = pd.DataFrame({
    "file": test_names,
    "label": labels_cnn
})
df_cnn.to_csv("predicciones_cnn.csv", index=False)
print("generado correctamente")


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
✅ CSV de CNN guardado como 'predicciones_cnn.csv'


## Comparación de Accuracy

In [30]:
print("Accuracy de regresion logistica:", accuracy_score(y_val, y_pred_lr))
print("Accuracy de red convolucional:", accuracy_score(y_val, y_pred_cnn_labels))


Accuracy de regresión logística: 0.4436229205175601
Accuracy de red convolucional: 0.7181146025878004


La red convolucional demostró un desempeño superior frente al modelo de regresion logistica.